In [1]:
import pandas as pd

COLUMNS = [ 'Polyuria', 'Polydipsia', 'sudden weight loss',
       'weakness', 'Polyphagia', 'Genital thrush', 'visual blurring',
       'Itching', 'Irritability', 'delayed healing', 'partial paresis',
       'muscle stiffness', 'Alopecia', 'Obesity',]
DATA = '/kaggle/input/early-stage-diabetes-risk-prediction/diabetes_data_upload.csv'
TARGET = 'class'

df = pd.read_csv(filepath_or_buffer=DATA)
df['Gender'] = df['Gender'] == 'Male'
for column in COLUMNS:
    df[column] = df[column] == 'Yes'
df[TARGET] = df[TARGET] == 'Positive'
df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,True,False,True,False,True,False,False,False,True,False,True,False,True,True,True,True
1,58,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True
2,41,True,True,False,False,True,True,False,False,True,False,True,False,True,True,False,True
3,45,True,False,False,True,True,True,True,False,True,False,True,False,False,False,False,True
4,60,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True
